**_<div style="text-align: center"> - - -   TEAM 4 | SMU Data Science Bootcamp   - - -</div>_**

In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Extract CSVs into DataFrames 

In [2]:
#Extract CSV into DataFrame
overdose_file = "overdoses.csv"
overdose_file_df = pd.read_csv(overdose_file)
overdose_file_df.head()

,State,Population,Deaths,Abbrev
0,Alabama,"4,833,722",723,AL
1,Alaska,"735,132",124,AK
2,Arizona,"6,626,624","1,211",AZ
3,Arkansas,"2,959,373",356,AR
4,California,"38,332,521","4,521",CA


In [3]:
#Extract CSV into DataFrame
prescriber_info_file = "prescriber-info.csv"
prescriber_info_file_df_org = pd.read_csv(prescriber_info_file)
prescriber_info_file_df_org.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [4]:
#Extract CSV into DataFrame
age_adjusted_rate = "raw_data.csv"
age_adjusted_rate_df_org = pd.read_csv(age_adjusted_rate)
age_adjusted_rate_df_org.head()

,Location,Opioid Overdose Death Rate (Age-Adjusted),All Drug Overdose Death Rate (Age-Adjusted),Percent Change in Opioid Overdose Death Rate from Prior Year,Percent Change in All Drug Overdose Death Rate from Prior Year
0,United States,9.0,14.7,0.14,0.07
1,Alabama,5.6,15.2,0.6,0.20
2,Alaska,10.6,16.8,0.15,0.17
3,Arizona,8.8,18.2,0.07,-0.03
4,Arkansas,6.3,12.6,0.13,0.14


### Clean DataFrames
#### Select the columns 
All the columns in overdoses_file_df is kept.

In [5]:
# Create new data with select columns
prescriber_info_file_df = prescriber_info_file_df_org[['State','Gender', 'Specialty']]
prescriber_info_file_df.head()

,State,Gender,Specialty
0,TX,M,Dentist
1,AL,F,General Surgery
2,NY,F,General Practice
3,AZ,M,Internal Medicine
4,NV,M,Hematology/Oncology


Drop first row of the age adjusted rate DataFrame 

In [6]:
age_adjusted_rate_df = age_adjusted_rate_df_org.iloc[1:]
age_adjusted_rate_df.head()

,Location,Opioid Overdose Death Rate (Age-Adjusted),All Drug Overdose Death Rate (Age-Adjusted),Percent Change in Opioid Overdose Death Rate from Prior Year,Percent Change in All Drug Overdose Death Rate from Prior Year
1,Alabama,5.6,15.2,0.6,0.20
2,Alaska,10.6,16.8,0.15,0.17
3,Arizona,8.8,18.2,0.07,-0.03
4,Arkansas,6.3,12.6,0.13,0.14
5,California,5.0,11.1,0.02,0.00


#### Check for duplicate values in the DataFrames

In [7]:
#Check for the shape of the DataFrame
overdose_file_df.shape

(50, 4)

In [8]:
prescriber_info_file_df.shape

(25000, 3)

In [9]:
# Check for duplicates
overdose_file_df = overdose_file_df.drop_duplicates(subset=None, keep='first', inplace= False)

#### Check for null values in the DataFrames

In [10]:
# Check for null values
overdose_file_df.isnull().values.any()

False

In [11]:
# Check for null values
prescriber_info_file_df.isnull().values.any()

False

In [12]:
# Check for null values
age_adjusted_rate_df.isnull().values.any()

True

In [13]:
# Drop null values
age_adjusted_rate_df = age_adjusted_rate_df.dropna()

#### Change Column names for the DataFrames
Since "State" is a key in SQL and gives error

In [14]:
overdose_file_df_renamed = overdose_file_df.rename(columns={'State': 'state_name', 'Population': 'population','Deaths': 'deaths', 'Abbrev': 'abbrev'})

In [15]:
prescriber_info_file_df_renamed = prescriber_info_file_df.rename(columns={'State': 'state_name', 'Gender': 'gender', 'Specialty': 'specialty' })

In [16]:
age_adjusted_rate_df_renamed = age_adjusted_rate_df.rename(columns={'Location': 'state_name','Opioid Overdose Death Rate (Age-Adjusted)': 'opioid_death_rate','All Drug Overdose Death Rate (Age-Adjusted)': 'all_overdose_rate','Percent Change in Opioid Overdose Death Rate from Prior Year' :'opioid_percent_change','Percent Change in All Drug Overdose Death Rate from Prior Year': 'overdose_percent_change' })

### Connect to local database

In [17]:
# Import postgreSQL password
from config import postgres_pass

# Use your own username and password in the following code accordingly
# rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"

rds_connection_string = (f"postgres:{postgres_pass}@localhost:5432/Team4_ETL_db")
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [18]:
engine.table_names()

['states', 'overdose', 'prescriber', 'age_adjusted_rate']

### Use pandas to load csv converted DataFrame into database

In [19]:
overdose_file_df_renamed.to_sql(name='overdose', con=engine, if_exists='append', index=False)

In [20]:
prescriber_info_file_df_renamed.to_sql(name='prescriber', con=engine, if_exists='append', index=False)

In [21]:
age_adjusted_rate_df_renamed.to_sql(name='age_adjusted_rate', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table

In [22]:
pd.read_sql_query('select * from overdose', con=engine).head()

,state_name,population,deaths,abbrev
0,Alabama,"4,833,722",723,AL
1,Alaska,"735,132",124,AK
2,Arizona,"6,626,624","1,211",AZ
3,Arkansas,"2,959,373",356,AR
4,California,"38,332,521","4,521",CA


In [23]:
pd.read_sql_query('select * from prescriber', con=engine).head()

,state_name,gender,specialty
0,TX,M,Dentist
1,AL,F,General Surgery
2,NY,F,General Practice
3,AZ,M,Internal Medicine
4,NV,M,Hematology/Oncology


In [24]:
pd.read_sql_query('select * from age_adjusted_rate', con=engine).head()

,state_name,opioid_death_rate,all_overdose_rate,opioid_percent_change,overdose_percent_change
0,Alabama,5.6,15.2,0.6,0.2
1,Alaska,10.6,16.8,0.15,0.17
2,Arizona,8.8,18.2,0.07,-0.03
3,Arkansas,6.3,12.6,0.13,0.14
4,California,5.0,11.1,0.02,0.0
